In [1]:
#Latin Hypercube Sampling 

In [1]:
import bpy
import subprocess
import json
import os
import random
from pathlib import Path

## Getting the position of the armature from Blender

This function 
- reaches out to the specified blender file
- returns the position of the armature

In [2]:
def get_armature_from_blender(blend_file_path):
    blender_executable_path = '/Applications/Blender3.4.1.app/Contents/MacOS/Blender'     # Mac
    # blender_executable_path = ''     # Windows
    script_file_path = 'utils/get_armature.py'
    command = [
        blender_executable_path,
        '-b', blend_file_path,
        '-P', script_file_path,
        '--',
    ]
    
    result = subprocess.run(command, capture_output=True, text=True)
    output_lines = result.stdout.split('\n')
    json_data = ''
    recording = False
    for line in output_lines:
        if line == '---JSON_START---':
            recording = True
        elif line == '---JSON_END---':
            break
        elif recording:
            json_data += line

    try:
        if json_data:
            armature_data = json.loads(json_data)
            return armature_data
        else:
            print("No armature data was found.")
            return None
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON from Blender output: {e}")
        return None

## Applying the Perturbation

This function...
- changes the rig in a random way
- based on the constraints it is given in armature_boundaries.json
- the idle/base configuration is being taken from armature_data.json

In [3]:
def perturbate_armature_in_blender(blend_file_path):
    blender_executable_path = '/Applications/Blender3.4.1.app/Contents/MacOS/Blender' # Mac
    # blender_executable_path = '' # Windows    
    script_file_path = '/Users/leonpaletta/Coding/MT/root/render/utils/perturbate_armature.py'  
    path_of_idle_armature = 'idle_armature.json'
    
    command = [
        blender_executable_path,
        '-b', blend_file_path,
        '--python', script_file_path,
        '--', path_of_idle_armature
    ]
    subprocess.run(command)

## Rendering the scene in Blender

In [4]:
def render_blend_file(blend_file_path, output_folder, out_file_name, running_number, current_rig_params):
    os.makedirs(output_folder, exist_ok=True)
    blender_executable_path = '/Applications/Blender3.4.1.app/Contents/MacOS/Blender' # Mac
    # blender_executable_path = '' # Windows
    output_file_path = os.path.join(output_folder, f"{running_number}_{out_file_name}")
    params_json_path = os.path.join(output_folder, 'rendering_rig_params.json')

    py_command = "import bpy; bpy.context.scene.render.resolution_x=512; bpy.context.scene.render.resolution_y=512; bpy.context.scene.render.resolution_percentage=100"

    command = [
        blender_executable_path,        # Path to the Blender executable
        '-b', blend_file_path,          # Specify the .blend file
        '--python-expr', py_command,    # Run the Python command to set resolution
        '-o', output_file_path,         # Set the output file path
        '-F', 'PNG',                    # Set the output format to PNG
        '-f', '1'                       # Render the first frame (change as needed)
    ]
    subprocess.run(command)    
    print(f"Render complete: {output_file_path}")

    if Path(params_json_path).exists():
        with open(params_json_path, 'r') as file:
            data = json.load(file)
    else:
        data = {}
    
    data[str(running_number)] = current_rig_params
    
    with open(params_json_path, 'w') as file:
        json.dump(data, file, indent=4) 

# Applying the shit

When running the first time with a new character, first run this, so that the idle armature is being saved:

In [5]:
path_to_blender_file = '/Users/leonpaletta/Coding/MT/root/characters/blender_base/ulf.blend'
first_time = False


bones_data = get_armature_from_blender(path_to_blender_file)
print("The idle position of the relevant bones in the armature is: ")
print(bones_data)
if first_time:
    output_file='idle_armature.json'
    with open(output_file, 'w') as outfile:
        json.dump(bones_data, outfile, indent=4)
    print("Saved the idle armature to idle_armature.json")

The idle position of the relevant bones in the armature is: 
{'jaw_master': {'location': [0.07955741882324219, -7.746394634246826, 160.69256591796875], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'eyes': {'location': [0.07955741882324219, -33.89311218261719, 162.1348876953125], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'eye.R': {'location': [-3.5866665840148926, -33.89311218261719, 162.1348876953125], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'eye.L': {'location': [3.745781421661377, -33.89311218261719, 162.1348876953125], 'rotation': [1.0, 0.0, 0.0, 0.0]}}


In [42]:
perturbate_armature_in_blender(path_to_blender_file)
get_armature_from_blender(path_to_blender_file)

Reloading external rigs...
Reloading external metarigs...
Info: Saved "ulf.blend"
Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:48:52)
Read prefs: /Users/leonpaletta/Library/Application Support/Blender/3.4/config/userpref.blend
Read blend: /Users/leonpaletta/Coding/MT/root/characters/blender_base/ulf.blend

Blender quit


{'jaw_master': {'location': [-1.3507120609283447,
   -6.2061614990234375,
   161.39935302734375],
  'rotation': [1.0,
   0.026017513126134872,
   -0.0071640172973275185,
   -0.0944049209356308]},
 'eyes': {'location': [-0.806123673915863,
   -33.98600769042969,
   160.32020568847656],
  'rotation': [1.0,
   -8.419555342698004e-06,
   -7.307767191377934e-06,
   6.839448815298965e-06]},
 'eye.R': {'location': [-5.05916690826416,
   -33.93577194213867,
   160.699462890625],
  'rotation': [1.0,
   -9.12790528673213e-06,
   3.6711014672619058e-06,
   5.0047756303683855e-06]},
 'eye.L': {'location': [2.0793166160583496,
   -33.632049560546875,
   159.5586700439453],
  'rotation': [1.0,
   2.7371345368010225e-06,
   -9.98590167000657e-06,
   9.8491027529235e-06]}}

# Data Generator
Creates samples...
- in the predefined range specified in armature_boundaries.json
- takes as idle position the information from idle_armature.json
- takes into consideration all the bones specified in relevant_armature.json 

In [6]:
n_samples = 64

In [7]:
# dont change anything, just run this first and change the name to 'idle.png', which will then be the reference picture
rig_params = get_armature_from_blender(path_to_blender_file)
render_blend_file(blend_file_path = path_to_blender_file, output_folder = 'output/', out_file_name = 'rendis', running_number = str(i), current_rig_params = rig_params)

NameError: name 'i' is not defined

In [8]:
for i in range(1, n_samples): 
    perturbate_armature_in_blender(path_to_blender_file)
    rig_params = get_armature_from_blender(path_to_blender_file)
    render_blend_file(blend_file_path = path_to_blender_file, output_folder = 'output/', out_file_name = 'rendis', running_number = str(i), current_rig_params = rig_params)

Reloading external rigs...
Reloading external metarigs...
Info: Saved "ulf.blend"
Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:48:52)
Read prefs: /Users/leonpaletta/Library/Application Support/Blender/3.4/config/userpref.blend
Read blend: /Users/leonpaletta/Coding/MT/root/characters/blender_base/ulf.blend

Blender quit
Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:48:52)
Read prefs: /Users/leonpaletta/Library/Application Support/Blender/3.4/config/userpref.blend
Read blend: /Users/leonpaletta/Coding/MT/root/characters/blender_base/ulf.blend
Fra:1 Mem:87.55M (Peak 88.68M) | Time:00:00.61 | Syncing Light
Fra:1 Mem:87.55M (Peak 88.68M) | Time:00:00.61 | Syncing Camera
Fra:1 Mem:87.56M (Peak 88.68M) | Time:00:00.61 | Syncing Ch22_Hair
Fra:1 Mem:120.75M (Peak 135.59M) | Time:00:01.11 | Syncing Ch22_Body
Fra:1 Mem:378.07M (Peak 632.80M) | Time:00:02.61 | Syncing rig
Fra:1 Mem:378.08M (Peak 632.80M) | Time:00:02.61 | Syncing Ch22_Eyelashes
Fra:1 Mem:378.16M (Peak 632.80M) | Tim